In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D
from keras.optimizers.legacy import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

In [ ]:
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
device = "GPU"
physicalDevices = tf.config.experimental.list_physical_devices(device)
print(f"available GPU count: {len(physicalDevices)}")
if len(physicalDevices) > 0:
    tf.config.experimental.set_memory_growth(physicalDevices[0])

In [ ]:
os.chdir("data/dogs-vs-cats")
force = True
if os.path.isdir("train/cat") is False:
    os.makedirs("train/cat")
    os.makedirs("valid/cat")
    os.makedirs("test/cat")
    os.makedirs("train/dog")
    os.makedirs("valid/dog")
    os.makedirs("test/dog")
    for image in random.sample(glob.glob("train/cat*"), 500):
        shutil.move(image, "train/cat")
    for image in random.sample(glob.glob("train/cat*"), 100):
        shutil.move(image, "valid/cat")
    for image in random.sample(glob.glob("train/cat*"), 50):
        shutil.move(image, "test/cat")
    for image in random.sample(glob.glob("train/dog*"), 500):
        shutil.move(image, "train/dog")
    for image in random.sample(glob.glob("train/dog*"), 100):
        shutil.move(image, "valid/dog")
    for image in random.sample(glob.glob("train/dog*"), 50):
        shutil.move(image, "test/dog")
    os.chdir("../../")

In [ ]:
trainPath = "data/dogs-vs-cats/train"
validPath = "data/dogs-vs-cats/valid"
testPath = "data/dogs-vs-cats/test"
if "valid" in os.listdir():
    trainPath = "train"
    validPath = "valid"
    testPath = "test"
os.listdir()

In [ ]:
trainBatches = ImageDataGenerator(
    preprocessing_function=keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=trainPath,
    target_size=(244, 244),
    classes=["cat", "dog"],
    batch_size=10,
    shuffle=True
)
validBatches = ImageDataGenerator(
    preprocessing_function=keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=validPath,
    target_size=(244, 244),
    classes=["cat", "dog"],
    batch_size=10,
    shuffle=True
)
testBatches = ImageDataGenerator(
    preprocessing_function=keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=testPath,
    target_size=(244, 244),
    classes=["cat", "dog"],
    batch_size=10,
    shuffle=False
)

In [ ]:
assert trainBatches.n == 1000
assert validBatches.n == 200
assert testBatches.n == 100
assert trainBatches.num_classes == 2
assert validBatches.num_classes == 2
assert testBatches.num_classes == 2

In [ ]:
def plotImages(imageArray):
    figure, axis = plt.subplots(1, 10, figsize =(20, 20))
    axis = axis.flatten()
    for image, ax in zip(imageArray, axis):
        ax.imshow(image)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
# images, labels = next(trainBatches)
# plotImages(images)
# print(labels)

In [ ]:
model = Sequential([
    Conv2D(
        filters=32, 
        kernel_size=(3, 3), 
        activation='relu', 
        padding='same', 
        input_shape=(244, 244, 3)
    ),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units=2, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.0001), 
    loss="categorical_crossentropy", 
    metrics=["accuracy"]
)

In [ ]:
model.fit(x=trainBatches, validation_data=validBatches, epochs=10, verbose=2)

In [ ]:
model.save("dogs_vs_cats.h5")